In [1]:
#same as process_labels2, but a binary problem, just high and low risk
import torch
def process_labels3(o_labels):
    l1 = o_labels["p1_label"]
    l2 = o_labels["p2_label"]
    l3 = o_labels["p3_label"]
    l = [l1, l2, l3]
    #choose most severe diagnosis
    if "ADH" in l:
        if l.count("ADH") == 3:
            return torch.tensor([1, 1])
        elif l.count("ADH") == 2:
            return torch.tensor([1, 0.66])
        else:
            return torch.tensor([1, 0.33])

    elif "Flat Epithelial" in l:
        if l.count("Flat Epithelial") == 3:
            return torch.tensor([1, 1])
        elif l.count("Flat Epithelial") == 2:
            return torch.tensor([1, 0.66])
        else:
            return torch.tensor([1, 0.33])

    elif "Columnar" in l:
        if l.count("Columnar") == 3:
            return torch.tensor([0, 1])
        elif l.count("Columnar") == 2:
            return torch.tensor([0, 0.66])
        else:
            return torch.tensor([0, 0.33])

    elif "Normal Duct" in l:
        if l.count("Normal Duct") == 3:
            return torch.tensor([0, 1])
        elif l.count("Normal Duct") == 2:
            return torch.tensor([0, 0.66])
        else:
            return torch.tensor([0, 0.33])     

    else:
        return torch.tensor([-1, 0])

In [2]:
#other option, just always take the most severe diagnosis, use count for confidence (hopefully get more high risk examples)
def process_labels2(o_labels):
    l1 = o_labels["p1_label"]
    l2 = o_labels["p2_label"]
    l3 = o_labels["p3_label"]
    l = [l1, l2, l3]
    #choose most severe diagnosis
    if "ADH" in l:
        if l.count("ADH") == 3:
            return torch.tensor([3, 1])
        elif l.count("ADH") == 2:
            return torch.tensor([3, 0.66])
        else:
            return torch.tensor([3, 0.33])

    elif "Flat Epithelial" in l:
        if l.count("Flat Epithelial") == 3:
            return torch.tensor([2, 1])
        elif l.count("Flat Epithelial") == 2:
            return torch.tensor([2, 0.66])
        else:
            return torch.tensor([2, 0.33])

    elif "Columnar" in l:
        if l.count("Columnar") == 3:
            return torch.tensor([1, 1])
        elif l.count("Columnar") == 2:
            return torch.tensor([1, 0.66])
        else:
            return torch.tensor([1, 0.33])

    elif "Normal Duct" in l:
        if l.count("Normal Duct") == 3:
            return torch.tensor([0, 1])
        elif l.count("Normal Duct") == 2:
            return torch.tensor([0, 0.66])
        else:
            return torch.tensor([0, 0.33])     

    else:
        return torch.tensor([-1, 0])

In [10]:
#set up dataset, just lists of graphs defined as edge pair list and node attribute dict
#read all the jsons and add them to Data list
import json
import os
import math
import numpy as np

json_dir= r"C:\Users\Brian\Desktop\AIProject\roi_graph_jsons"
G = []
y = []
for roi_json in os.listdir(json_dir):
    with open(os.path.join(json_dir, roi_json), 'r') as f:
        roi_data = json.loads(f.read())
        #Json has following fields:
        #  name - name of the image
        #  node_features - [num_nodes num_features] matrix of features for each node
        #  dist_edge_table - [2 num_edges] edges defined by distance metric
        #  labels - dict, which contains p1_label, p2_label, p3_label (roi_data["labels"]["p1_label"]
        
        #process the labels. see the function for more details
        label, conf = process_labels2(roi_data["labels"])
        if label == -1:
            #print("Bad sample, no label. Discarding.")
            continue
        
        y.append(label.item())
        
        g = []  #represents a single graph
        edges = set()         #set of edge pairs
        for i in range(len(roi_data["dist_edge_table"][0])):
            n1 = roi_data["dist_edge_table"][0][i] + 1
            n2 = roi_data["dist_edge_table"][1][i] + 1
            edges.add((n1, n2))
        
        g.append(edges)
        x = {}   #node attributes
        for i in range(len(roi_data["node_features"])):
            x.update({i+1 : roi_data["node_features"][i]})
        g.append(x)
        g.append({})#example did this
        G.append(g)
        
#print(G[0])

In [11]:
#actual model

from __future__ import print_function
print(__doc__)

import numpy as np
import pdb

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from grakel.kernels import PropagationAttr

y = np.int_(y)
# Splits the dataset into a training and a test set
G_train, G_test, y_train, y_test = train_test_split(G, y, test_size=0.2, random_state=42)

#resample training so we have even class representation
y_train_inv = np.int_(np.ones(len(y_train))-y_train)
pos_inds = np.nonzero(y_train)[0]
neg_inds = np.nonzero(y_train_inv)[0]

pos_inds = np.random.choice(pos_inds, len(neg_inds))
G_train_t = []
y_train_t = []
for i in range(len(pos_inds)):
    G_train_t.append(G_train[pos_inds[i]])
    y_train_t.append(y_train[pos_inds[i]])
for i in range(len(neg_inds)):
    G_train_t.append(G_train[neg_inds[i]])
    y_train_t.append(y_train[neg_inds[i]])
#y_train = y_train_t
#G_train = G_train_t

# Uses the graphhopper kernel to generate the kernel matrices
gk = PropagationAttr(normalize=True)
K_train = gk.fit_transform(G_train)
K_test = gk.transform(G_test)
print("Done graph propagation")

Automatically created module for IPython interactive environment
Done graph propagation


In [12]:
#force test data to be equal class representation
y_test_inv = np.int_(np.ones(len(y_test))-y_test)
pos_inds = np.nonzero(y_test)[0]
neg_inds = np.nonzero(y_test_inv)[0]

pos_inds = np.random.choice(pos_inds, len(neg_inds))
G_test_t = []
y_test_t = []
for i in range(len(pos_inds)):
    G_test_t.append(G_test[pos_inds[i]])
    y_test_t.append(y_test[pos_inds[i]])
for i in range(len(neg_inds)):
    G_test_t.append(G_test[neg_inds[i]])
    y_test_t.append(y_test[neg_inds[i]])
#y_test = y_test_t
#G_test = G_test_t

K_test = gk.transform(G_test)

In [13]:
# Uses the SVM classifier to perform classification
clf = SVC(kernel="precomputed", probability=True)
clf.fit(K_train, y_train)
y_pred_probs = clf.predict_proba(K_test)
y_pred = np.argmax(y_pred_probs, 1)

# Computes and prints the classification accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", str(round(acc*100, 2)) + "%")

Accuracy: 52.01%


In [14]:
from sklearn.metrics import confusion_matrix, f1_score, classification_report, confusion_matrix
from sklearn.metrics import auc
(len(y_test) - np.count_nonzero(y_test)) / (len(y_test))
print("Confusion matrix (known=rows, predicted=cols)")
#print(confusion_matrix(y_test,y_pred))
#print(len(y_pred))
#print("f1 score: " + str(f1_score(y_test, y_pred)))

report = classification_report(y_test, y_pred)
print(report)
print(confusion_matrix(y_test,y_pred))

#print("AUC: " + str(auc(y_pred_probs, y_pred)))

Confusion matrix (known=rows, predicted=cols)
              precision    recall  f1-score   support

           0       0.54      0.95      0.69       169
           1       0.00      0.00      0.00        59
           2       0.21      0.07      0.10        44
           3       0.42      0.10      0.16        51

    accuracy                           0.52       323
   macro avg       0.29      0.28      0.24       323
weighted avg       0.38      0.52      0.40       323

[[160   0   5   4]
 [ 52   0   5   2]
 [ 40   0   3   1]
 [ 45   0   1   5]]


c:\users\brian\anaconda3\envs\pytorch_geometric\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
#compute average confidence scores over the classes
avg_hr = 0
num_hr = 0
avg_lr = 0
num_lr = 0
for i in range(len(y_pred)):
    if y_pred[i] == 0: #low risk
        avg_lr += y_pred_probs[i][0]
        num_lr += 1
    else:
        avg_hr += y_pred_probs[i][1]
        num_hr += 1
        
print("Averge low risk score: " + str(avg_lr / num_lr))
print("Average high risk score: " + str(avg_hr / num_hr))

Averge low risk score: 0.9148334882415966
Average high risk score: 0.749013617292631
